### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# 1. Genera i dati
np.random.seed(0)
x = np.linspace(-3, 3, 100)
y = x**3 - x + np.random.normal(0, 3, size=x.shape)

# App Dash
app = dash.Dash(__name__)
app.title = "Regressione polinomiale interattiva"

app.layout = html.Div([
    html.H2("Regressione polinomiale interattiva"),
    dcc.Slider(
        id='grado_slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: f"Grado {i}" for i in range(1, 11)}
    ),
    dcc.Graph(id='grafico')
])

@app.callback(
    Output('grafico', 'figure'),
    Input('grado_slider', 'value')
)
def aggiorna_grafico(grado):
    # 2. Modello polinomiale
    X = x.reshape(-1, 1)
    poly = PolynomialFeatures(degree=grado)
    X_poly = poly.fit_transform(X)
    
    model = LinearRegression()
    model.fit(X_poly, y)
    y_pred = model.predict(X_poly)

    # 3. Grafico con Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Dati reali'))
    fig.add_trace(go.Scatter(x=x, y=y_pred, mode='lines', name=f'Polinomio grado {grado}'))
    fig.update_layout(
        xaxis_title='x',
        yaxis_title='y',
        title=f"Regressione polinomiale - Grado {grado}"
    )
    return fig

# Avvia il server
if __name__ == '__main__':
    app.run(debug=True)
